In [1]:
import pysam
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
import gc

In [2]:
class Correction_Summary:
    
    def __init__(self, bam, bam_corr, method):
        self.bam = bam
        self.bam_corr = bam_corr
        self.method = method
        self.table = np.zeros((2, 3))   
        self.mismapped = 0
        self.failed_corr = 0
        self.failed_raw = 0
        
    def errors_in_corr(self):
        
        self.corr_errors = {}
        
        with pysam.AlignmentFile(self.bam_corr, 'rb') as al_corr:
            for read in tqdm_notebook(al_corr.fetch()):
                if self.method == 'BH':
                    
                    temp = {}
                    temp['anchors'] = (read.reference_start, read.reference_end)
                    temp['is_rev'] = read.is_reverse
                    temp['errors'] = {}
                    temp['trimmed'] = {}
                    
                    try:
                        for qb, rb, base in read.get_aligned_pairs(with_seq=True):

                            if (qb != None):
                                if (read.seq[qb] == 'N'):
                                    temp['trimmed'][qb] = (rb)
                                elif (read.seq[qb] != base):
                                    temp['errors'][qb] = (rb, base)
                                else:
                                    continue
                    except:
                        self.failed_corr += 1
                    
                    self.corr_errors[f'{read.qname}{read.is_read1}'] = temp
                    
                
    def compute_results(self):
        with pysam.AlignmentFile(self.bam, 'rb') as al:
            for read in tqdm_notebook(al.fetch()):
                if f'{read.qname}{read.is_read1}' in self.corr_errors:
                    
                    t = self.corr_errors[f'{read.qname}{read.is_read1}']
                    
                    if (t['is_rev'] == read.is_reverse):
                        if (read.reference_start == t['anchors'][0])\
                        or (read.reference_end == t['anchors'][1]):
                            try:
                                for qb, rb, base in read.get_aligned_pairs(with_seq = True):
                                    if (qb != None):

                                        if (rb == None) or (base.upper() != base):
                                            row = 0
                                        else:
                                            row = 1

                                        if qb in t['trimmed']:
                                            col = 2
                                        elif qb in t['errors']:
                                            col = 0
                                        else:
                                            col = 1

                                        self.table[row, col] += 1
                            except:
                                self.failed_raw += 1
                                   
                        else:
                            self.mismapped += 1
                            continue
                    else:
                        self.mismapped += 1
                        continue
        
    def fit(self):
        self.errors_in_corr()
        self.compute_results()
        self.table = pd.DataFrame(self.table,
                                  index = ['Error in raw', 'Correct in raw'],
                                 columns= ['Error in corrected', 'Correct in corrected', 'Removed'])
        return self.table

## Test data:

In [9]:
bam_path_pre = 'Data/Assignment_4/test_al/s_pre_corr.bam'
bam_path_post = 'Data/Assignment_4/test_al/s_post_corr.bam'

In [10]:
Summary = Correction_Summary(bam_path_pre, bam_path_post, 'BH')
test_res = Summary.fit()

In [11]:
test_res

,Error in corrected,Correct in corrected,Removed
Error in raw,6476.0,21244.0,200904.0
Correct in raw,1570.0,5222528.0,345578.0


In [ ]:
print('Number of reads aligned differently before and afrer correction: {}'.format(Summary.mismapped))

## Full data:

In [3]:
bam_path_r_pre = 'Data/Assignment_4/real_al/s_pre_corr.bam'
bam_path_r_post = 'Data/Assignment_4/real_al/s_post_corr.bam'

In [4]:
Summary_r = Correction_Summary(bam_path_r_pre, bam_path_r_post, 'BH')
test_res_r = Summary_r.fit()

In [6]:
test_res_r

,Error in corrected,Correct in corrected,Removed
Error in raw,382856.0,926838.0,6687031.0
Correct in raw,77557.0,246947772.0,14383146.0


In [5]:
print('Number of reads aligned differently before and afrer correction: {}'.format(Summary_r.mismapped))

Number of reads aligned differently before and afrer correction: 21942
